In [1]:
import pandas as pd
import numpy as np
import sklearn
from sklearn import preprocessing 
import regex as re

In [23]:
#!pip install regex

^C


In [2]:
df=pd.read_csv("sales_train.csv")
df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day
0,02.01.2013,0,59,22154,999.00,1.0
1,03.01.2013,0,25,2552,899.00,1.0
2,05.01.2013,0,25,2552,899.00,-1.0
3,06.01.2013,0,25,2554,1709.05,1.0
4,15.01.2013,0,25,2555,1099.00,1.0


In [3]:
def add_datepart(df, fldname, drop=True, time=False):
    "Helper function that adds columns relevant to a date."
    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [4]:
add_datepart(df,"date")

<ipython-input-3-eb3dfbe05590>:14: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())


In [26]:
df.head()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed
0,0,59,22154,999.00,1.0,2013,2,5,1,4,32,False,True,False,False,False,False,1359676800
1,0,25,2552,899.00,1.0,2013,3,9,1,4,60,False,True,False,False,False,False,1362096000
2,0,25,2552,899.00,-1.0,2013,5,18,1,2,121,False,True,False,False,False,False,1367366400
3,0,25,2554,1709.05,1.0,2013,6,22,1,5,152,False,True,False,False,False,False,1370044800
4,0,25,2555,1099.00,1.0,2013,1,3,15,1,15,False,False,False,False,False,False,1358208000


In [13]:
df['Is_month_end']

0          False
1          False
2          False
3          False
4          False
           ...  
2935844    False
2935845    False
2935846    False
2935847    False
2935848    False
Name: Is_month_end, Length: 2935849, dtype: bool

In [5]:
label_encoder = preprocessing.LabelEncoder()

In [6]:
df['Is_month_end']= label_encoder.fit_transform(df['Is_month_end']) 
df['Is_month_start']= label_encoder.fit_transform(df['Is_month_start']) 
df['Is_quarter_end']= label_encoder.fit_transform(df['Is_quarter_end']) 
df['Is_quarter_start']= label_encoder.fit_transform(df['Is_quarter_start']) 
df['Is_year_end']= label_encoder.fit_transform(df['Is_year_end']) 
df['Is_year_start']= label_encoder.fit_transform(df['Is_year_start']) 

In [7]:
item_cnt=df['item_cnt_day']
df=df.drop(labels='item_cnt_day',axis=1)

In [8]:
df.insert(17,'item_cnt_day',item_cnt);

In [9]:
df.head()

,date_block_num,shop_id,item_id,item_price,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,item_cnt_day
0,0,59,22154,999.00,2013,2,5,1,4,32,0,1,0,0,0,0,1359676800,1.0
1,0,25,2552,899.00,2013,3,9,1,4,60,0,1,0,0,0,0,1362096000,1.0
2,0,25,2552,899.00,2013,5,18,1,2,121,0,1,0,0,0,0,1367366400,-1.0
3,0,25,2554,1709.05,2013,6,22,1,5,152,0,1,0,0,0,0,1370044800,1.0
4,0,25,2555,1099.00,2013,1,3,15,1,15,0,0,0,0,0,0,1358208000,1.0


In [13]:
df.iloc[0:2]

,date_block_num,shop_id,item_id,item_price,Year,Month,Week,Day,Dayofweek,Dayofyear,Is_month_end,Is_month_start,Is_quarter_end,Is_quarter_start,Is_year_end,Is_year_start,Elapsed,item_cnt_day
0,0,59,22154,999.0,2013,2,5,1,4,32,0,1,0,0,0,0,1359676800,1.0
1,0,25,2552,899.0,2013,3,9,1,4,60,0,1,0,0,0,0,1362096000,1.0


In [28]:
(5*len(df))/100

146792.45

In [29]:
2935849-146792

2789057

In [30]:
train_df=df[0:2789057]

In [31]:
valid_df=df[2789057:len(df)]

In [32]:
len(train_df),len(valid_df)

(2789057, 146792)

In [37]:
y_train=train_df['item_cnt_day']
x_train=train_df.drop('item_cnt_day',axis=1)

In [38]:
len(x_train),len(y_train)

(2789057, 2789057)

In [10]:
!pip install xgboost

In [11]:
import xgboost as xgb

In [12]:
from xgboost import XGBRegressor
xgb_model=XGBRegressor()

In [39]:
xgb_model.fit(x_train,y_train)

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "C:\Users\Personal\anaconda3\envs\py_38\lib\site-packages\IPython\core\interactiveshell.py", line 3343, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-39-e51a0bd71a19>", line 1, in <module>
    xgb_model.fit(x_train,y_train)
  File "C:\Users\Personal\anaconda3\envs\py_38\lib\site-packages\xgboost\sklearn.py", line 542, in fit
    self._Booster = train(params, train_dmatrix,
  File "C:\Users\Personal\anaconda3\envs\py_38\lib\site-packages\xgboost\training.py", line 208, in train
    return _train_internal(params, dtrain,
  File "C:\Users\Personal\anaconda3\envs\py_38\lib\site-packages\xgboost\training.py", line 75, in _train_internal
    bst.update(dtrain, i, obj)
  File "C:\Users\Personal\anaconda3\envs\py_38\lib\site-packages\xgboost\core.py", line 1159, in update
    _check_call(_LIB.XGBoosterUpdateOneIter(self.handle,
KeyboardInterrupt

During handling of the above exception, another exception occurr

TypeError: object of type 'NoneType' has no len()